# WMT17 ZH-EN Dataset Exploration

**For Google Colab:**
1. Mount Google Drive (run cell below)
2. Set `ROOT_DIR` to your project folder path
3. Run the rest of the notebook

**For local execution:** Skip the Google Drive cell and run from "Import Libraries"

In [ ]:
# Mount Google Drive (only needed for Google Colab)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # IMPORTANT: Set this to your code_zh_en directory path
    # This should point to where THIS notebook is located
    ROOT_DIR = "/content/drive/MyDrive/UofT/CSC2517/term_paper/code_zh_en"
    
    import os
    os.chdir(ROOT_DIR)
    print(f"✓ Changed to: {os.getcwd()}")
except ImportError:
    print("Not running on Colab, using local environment")

In [ ]:
# Verify working directory and data files
import os
print(f"Current directory: {os.getcwd()}")
print(f"\nContents of current directory:")
print(os.listdir('.'))

data_path = "../data/wmt17_zh_en_validation_2000"
if os.path.exists(data_path):
    print(f"\n✓ Data directory exists: {data_path}")
else:
    print(f"\n✗ Data directory NOT found: {data_path}")
    print("\nMake sure you:")
    print("1. Ran 01_load_data.py to download the data")
    print("2. Set ROOT_DIR to point to code_zh_en/ directory")
    print(f"\nExpected structure:")
    print(f"  ROOT_DIR/")
    print(f"  ├── data/")
    print(f"  │   └── wmt17_zh_en_validation_2000/")
    print(f"  └── <you are here>")

## Import Libraries

In [ ]:
# Import all required libraries
from datasets import load_from_disk
import pandas as pd
import random
import matplotlib.pyplot as plt
import pickle

In [ ]:
# Configure matplotlib for Chinese font support
# Check if running on Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # On Colab: Download and configure SimHei font for Chinese support
    import urllib.request
    import matplotlib
    import matplotlib.font_manager as fm
    import shutil
    
    # Download SimHei font from a reliable source
    font_url = "https://github.com/StellarCN/scp_zh/raw/master/fonts/SimHei.ttf"
    font_path = "/usr/share/fonts/truetype/SimHei.ttf"
    
    if not os.path.exists(font_path):
        print("Downloading SimHei font...")
        try:
            urllib.request.urlretrieve(font_url, font_path)
            print(f"✓ Downloaded font to {font_path}")
        except Exception as e:
            print(f"⚠ Download failed: {e}")
            # Try alternative source
            font_url = "https://github.com/kosho2013/DFPlayer/raw/master/SimHei.ttf"
            try:
                urllib.request.urlretrieve(font_url, font_path)
                print(f"✓ Downloaded from alternative source")
            except Exception as e2:
                print(f"⚠ Alternative download also failed: {e2}")
    else:
        print(f"✓ Font already exists at {font_path}")
    
    # Clear matplotlib cache
    cache_dir = matplotlib.get_cachedir()
    if os.path.exists(cache_dir):
        shutil.rmtree(cache_dir)
        print("✓ Cleared matplotlib cache")
    
    # Register the font if it exists
    if os.path.exists(font_path):
        fm.fontManager.addfont(font_path)
        print("✓ Font registered with matplotlib")
        
    # Set SimHei as default font
    plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False  # Fix minus sign display
    
    print("✓ Font configuration complete: SimHei")
    
else:
    # On local: use Arial Unicode MS (macOS)
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
    print("✓ Running locally: using Arial Unicode MS for Chinese support")

## 1. Load the saved dataset

In [ ]:
# Load the saved dataset
dataset = load_from_disk("../data/wmt17_zh_en_validation_2000")
print(f"Loaded {len(dataset)} sentence pairs")
print(f"\nFeatures: {dataset.features}")
print(f"Column names: {dataset.column_names}")

## 2. Convert to convenient formats

In [ ]:
# Convert to list of dictionaries (easiest for iteration)
data_list = []
for item in dataset:
    data_list.append({
        'english': item['translation']['en'],
        'chinese': item['translation']['zh']
    })

print(f"Created list with {len(data_list)} sentence pairs")

In [ ]:
# Convert to pandas DataFrame (good for analysis)
df = pd.DataFrame(data_list)
print(f"DataFrame shape: {df.shape}")
df.head()

## 3. Explore the data

In [ ]:
# Show random examples
print("="*80)
print("Random Examples:")
print("="*80)

for i in random.sample(range(len(data_list)), 5):
    print(f"\nExample {i}:")
    print(f"ZH: {data_list[i]['chinese']}")
    print(f"EN: {data_list[i]['english']}")

In [ ]:
# Compute sentence lengths
df['en_length'] = df['english'].str.split().str.len()
df['zh_length'] = df['chinese'].str.len()  # Chinese: character count since no spaces

print("Sentence Length Statistics:")
print("\nEnglish (words):")
print(df['en_length'].describe())
print("\nChinese (characters):")
print(df['zh_length'].describe())

In [ ]:
# Visualize length distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(df['en_length'], bins=30, alpha=0.7, color='blue', edgecolor='black')
axes[0].set_xlabel('Sentence Length (words)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('English Sentence Lengths')
axes[0].grid(alpha=0.3)

axes[1].hist(df['zh_length'], bins=30, alpha=0.7, color='red', edgecolor='black')
axes[1].set_xlabel('Sentence Length (characters)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Chinese Sentence Lengths')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Compare EN vs ZH lengths
plt.figure(figsize=(8, 6))
plt.scatter(df['en_length'], df['zh_length'], alpha=0.5)
plt.xlabel('English Length (words)')
plt.ylabel('Chinese Length (characters)')
plt.title('English vs Chinese Sentence Lengths')
plt.grid(alpha=0.3)
plt.show()

correlation = df['en_length'].corr(df['zh_length'])
print(f"\nCorrelation between EN and ZH lengths: {correlation:.3f}")

## 4. Export convenient formats for later use

In [ ]:
# Save as pickle for quick loading
with open('../data/sentence_pairs_zh_en.pkl', 'wb') as f:
    pickle.dump(data_list, f)

print("✓ Saved sentence pairs as pickle: ../data/sentence_pairs_zh_en.pkl")
print("\nTo load later: data_list = pickle.load(open('../data/sentence_pairs_zh_en.pkl', 'rb'))")

## Summary

**Available formats:**
- `dataset`: Original HuggingFace Dataset object
- `data_list`: List of dictionaries `[{'english': ..., 'chinese': ...}, ...]`
- `df`: Pandas DataFrame with columns ['english', 'chinese', 'en_length', 'zh_length']

**Exported files:**
- `sentence_pairs_zh_en.pkl`: Pickle format for quick loading in Python

**Next steps:**
1. Load NLLB-600M model
2. Extract attention maps
3. Build attention graphs
4. Compute persistent homology